In [1]:
import numpy as np
from pulp import *

In [2]:
# INPUT: 2D vector
z = np.array([[7, 1], [2, 2], [21, 3], [5, 5], [35, 3], [42, 20]])

In [3]:
# INPUT: FUNCTION
def myfunc(x):
    # This uses 2D
    return np.cos(x[:,0])**3+np.tanh(x[:,1])

In [4]:
# Results to replicate
y = myfunc(z)
list(y)

[1.1900885316571761,
 0.89196002432805166,
 0.83073195391416665,
 1.0227338651453095,
 0.25704583755905019,
 0.93600704854681505]

In [5]:
def random_params_to_fit_constraints(X):
    prob = LpProblem("Random Numbers to fit Constraints", LpMinimize)
    # Variables
    # A is in R(d)
    A = {}
    for j in range(X.shape[1]):
        A[j] = LpVariable("a_{}".format(j))
        
    # B is in R(n)
    B = {}
    for i in range(X.shape[0]):
        B[i] = LpVariable("beta_{}".format(i))
        
    # Interleaving Constraints
    # Tolerance for strict inequality
    tolerance = -5
    # b1 < x1 < b2 < x2 < b3 < x3 < b4 < x5
    for i in range(X.shape[0]):
        #b1 < x1, b2 < x2, b3 < x3, b4 < x5
        prob += B[i] <= lpSum( X[i,j] * A[j] for j in range(X.shape[1])) + tolerance
        # x1 < b2, x2 < b3, x3 < b4
        if i < X.shape[0]-1:
            prob += lpSum( X[i,j] * A[j] for j in range(X.shape[1])) <= B[i+1] + tolerance
            
    # Solve
    #print(prob)
    prob.solve()
    #print(LpStatus[prob.status])
    # Extract results
    if prob.status == 1:        
        A_opt = np.zeros(X.shape[1])
        B_opt = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            B_opt[i] = B[i].varValue
        for j in range(X.shape[1]):
            A_opt[j] = A[j].varValue
        X_opt = np.array([np.dot(X[i], A_opt) for i in range(X.shape[0])], dtype=np.float32)
        return A_opt, X_opt, B_opt
    else:
        return None

In [6]:
a, X, b = random_params_to_fit_constraints(z)
print(a)  # <a,z> = x
print(X)  # Transformed to R1
print(b)  # Bias

[  1.5  17.5]
[  28.   38.   84.   95.  105.  413.]
[   0.   33.   79.   90.  100.  408.]


In [7]:
# Constructed nxn matrix
A = np.transpose(np.ones((len(X), len(X)))*X) + np.ones((len(X), len(X)))*-b
A[A<0] = 0
A

array([[  28.,    0.,    0.,    0.,    0.,    0.],
       [  38.,    5.,    0.,    0.,    0.,    0.],
       [  84.,   51.,    5.,    0.,    0.,    0.],
       [  95.,   62.,   16.,    5.,    0.,    0.],
       [ 105.,   72.,   26.,   15.,    5.,    0.],
       [ 413.,  380.,  334.,  323.,  313.,    5.]])

In [8]:
# Weights matrix
w = np.linalg.solve(A, y) 
w

array([  4.25031618e-02,  -1.44632025e-01,   9.27339928e-01,
        -1.77706396e+00,   1.75056819e+00,  -4.90650696e+01])

In [9]:
np.allclose(np.dot(A, w), y)

True

In [10]:
# E.g, y1
w[0]*(X[0]-b[0]), y[0]

(1.1900885316571825, 1.1900885316571761)

In [11]:
# E.g, y2
(w[0]*(X[1]-b[0]))+(w[1]*(X[1]-b[1])), y[1]

(0.89196002432805221, 0.89196002432805166)

In [12]:
# Parameters:
print("2n + d = {}".format(z.shape[0]*2 + z.shape[1]))
print(len(w) + len(b) + len(a))

2n + d = 14
14
